In [5]:
import os
import shutil

import requests

import time
import numpy as np

import pandas as pd
import polars as pl
import polars.selectors as cs

import duckdb

import pyarrow.parquet as pq

## <p style='text-align: center; text-decoration: underline; color: #10A0B4;'> Datos oficiales de la **Union Europea** </p>
[Origen de los datos](https://www.eea.europa.eu/en/datahub/datahubitem-view/fa8b1229-3db6-495d-b18e-9c9b3267c02b)

---

## Autos registrados desde **2010** hasta **2023**

---

#### El dataset contiene mas de **16 Gigas** de informacion (80.378.486 filas).

El gran tamano del conjunto de datos trae problemas:
- No puedo subirlo a GitHub (+100MB). 
- Lento procesamiento.

Vamos a resolver ambas cuentiones achicando el dataset de manera tal que el impacto por usar menos informacion sea minimo

---

El archivo pesa +16GB y tiene más de 80 millones de filas. Tarda +3 min en ejecutar. 

In [6]:
url = 'ignore_data.csv'
file_weight = os.path.getsize(url)
print('Database size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

count = duckdb.sql("SELECT count(*) FROM ignore_data.csv").fetchone()[0]
print(f"El archivo .csv tiene {count} registros.")

Database size:  16.38 GB
El archivo .csv tiene 80378486 registros.


Tabla informacional de códigos y datatypes

<table>
<tbody>
<tr>
    <td>Name</td>
    <td>Definition</td>
    <td>Datatype </td>

</tr>
<tr>
    <td>ID</td>
    <td>Identification number.</td>
    <td>integer </td>
</tr>
<tr>
    <td>MS</td>
    <td>Member state.</td>
    <td>varchar(2) </td>
</tr>
<tr>
    <td>Mp</td>
    <td>Manufacturer pooling.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>VFN</td>
    <td>Vehicle family identification number.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Mh</td>
    <td>Manufacturer name EU standard denomination .</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>Man</td>
    <td>Manufacturer name OEM declaration.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>MMS</td>
    <td>Manufacturer name MS registry denomination .</td>
    <td>varchar(125) </td>
</tr>
<tr>
    <td>TAN</td>
    <td>Type approval number.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>T</td>
    <td>Type.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Va</td>
    <td>Variant.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Ve</td>
    <td>Version.</td>
    <td>varchar(35) </td>
</tr>
<tr>
    <td>Mk</td>
    <td>Make.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Cn</td>
    <td>Commercial name.</td>
    <td>varchar(50) </td>
</tr>
<tr>
    <td>Ct</td>
    <td>Category of the vehicle type approved.</td>
    <td>varchar(5)  </td>
</tr>
<tr>
    <td>Cr</td>
    <td>Category of the vehicle registered.</td>
    <td>varchar(5)  </td>
</tr>
<tr>
    <td>M (kg)</td>
    <td>Mass in running order Completed/complete vehicle .</td>
    <td>integer </td>
</tr>
<tr>
    <td>Mt</td>
    <td>WLTP test mass.</td>
    <td>integer</td>
</tr>
<tr>
    <td>Enedc (g/km)</td>
    <td>Specific CO2 Emissions (NEDC).</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ewltp (g/km)</td>
    <td>Specific CO2 Emissions (WLTP).</td>
    <td>integer </td>
</tr>
<tr>
    <td>W (mm)</td>
    <td>Wheel Base.</td>
    <td>integer </td>
</tr>
<tr>
    <td>At1 (mm)</td>
    <td>Axle width steering axle.</td>
    <td>integer </td>
</tr>
<tr>
    <td>At2 (mm)</td>
    <td>Axle width other axle.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ft</td>
    <td>Fuel type.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Fm</td>
    <td>Fuel mode.</td>
    <td>varchar(1)  </td>
</tr>
<tr>
    <td>Ec (cm3)</td>
    <td>Engine capacity.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Ep (KW)</td>
    <td>Engine power.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Z (Wh/km)</td>
    <td>Electric energy consumption.</td>
    <td>integer </td>
</tr>
<tr>
    <td>IT</td>
    <td>Innovative technology or group of innovative technologies.</td>
    <td>varchar(25) </td>
</tr>
<tr>
    <td>Ernedc (g/km)</td>
    <td>Emissions reduction through innovative technologies.</td>
    <td>float </td>
</tr>
<tr>
    <td>Erwltp (g/km)</td>
    <td>Emissions reduction through innovative technologies (WLTP).</td>
    <td>float </td>
</tr>
<tr>
    <td>De</td>
    <td>Deviation factor.</td>
    <td>float </td>
</tr>
<tr>
    <td>Vf</td>
    <td>Verification factor.</td>
    <td>integer </td>
</tr>
<tr>
    <td>R</td>
    <td>Total new registrations.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Year</td>
    <td>Reporting year.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Status</td>
    <td>P = Provisional data, F = Final data.</td>
    <td>varchar(1)  </td>
</tr>
<tr>
    <td>Version_file</td>
    <td>Internal versioning of deliverables.</td>
    <td>varchar(10) </td>
</tr>
<tr>
    <td>E (g/km)</td>
    <td>Specific CO2 Emission. Deprecated value, only relevant for data until 2016.</td>
    <td>float </td>
</tr>
<tr>
    <td>Er (g/km)</td>
    <td>Emissions reduction through innovative technologies. Deprecated value, only relevant for data until 2016.</td>
    <td>float </td>
</tr>
<tr>
    <td>Zr</td>
    <td>Electric range.</td>
    <td>integer </td>
</tr>
<tr>
    <td>Dr</td>
    <td>Registration date.</td>
    <td>date </td>
</tr>
<tr>
    <td>Fc</td>
    <td>Fuel consumption.</td>
    <td>float </td>
</tr>
</tbody>
</table>

---

<p style="font-size: 35px; text-align: center; color: yellow;"> Limpiado de datos </p>

<p style="font-size: 25px;">Crearé el DataFrame con la información cruda. Tal como la descargamos.</p>
<p style="font-size: 20px">Guardaré el DataFrame en archivo .parquet ya que internamente divide la información en bloques para que a la hora de procesarla vaya de a bloques de 1M registros por ejemplo.</p>

<p style="font-size: 25px">Para poder procesar los 80 millones de registros, usar <u>pandas</u> es inviable para mi computadora de 16GB de RAM. Provoca error 'MemoryError'</p>

- Pandas trabaja con el DF en memoria y a demás el DF debe de ocupar como máximo entre el 10% y 30% de la memoria RAM. 


<p>Se puede guardar el DataFrame en 80 archivos .parquet con 1M de registros cada uno o <u>en uno único</u> de 80M de registros. Usar <u>polars</u> o <u>duckdb</u> para procesar la información es mejor ya que ambos procesan archivos .parquet de a bloques (cargando y descargando en RAM los bloques .parquet) y a demás utilizan todos los núcleos de la CPU. </p>

- <b><u>Polars:</u></b> 
    - <b>Procesamiento:</b> Utiliza un motor escrito en Rust que procesa datos de forma paralela
    - <b>Sintaxis:</b> Utiliza expresiones parecidas a las de pandas pero más optimizadas. 
    - <b>Memoria:</b> Trabaja con "streaming" ("Lazy"), cargando y transformando sólolos trozos necesarios en cada momento. 

- <b><u>duckDB:</u></b>
    - <b>Procesamiento:</b> Es un motor SQL analítico que puede ejecutarse directamente en el script de Python.
    - <b>Flexibilidad:</b> Permite realizar transformaciones complejas mediante consultas SQL o su propia API de Python, siendo extremadamente eficiente para cruzar (join) múltiples archivos grandes.
    - <b>Out-of-Core:</b> Si los datos superan la RAM, DuckDB puede usar el disco duro como apoyo para que el proceso no falle.

<p style="font-size: 35px; text-align: center; color: orange;">Usaré las 2 librerías. POLARS para modificaciones del DF y DUCKDB para consultar.</p>

In [7]:
columns_to_load = ['ID', 'year', 'Country', 'Mh', 'z (Wh/km)', 'ep (KW)', 'Mk', 'Cn', 
                   'Electric range (km)', 'm (kg)', 'Ewltp (g/km)', 'Ft', 'ec (cm3)', 
                   'Fuel consumption ', 'MMS']

rename_columns = {
    'm (kg)': 'mass_kg', 'Ewltp (g/km)': 'co2_emission_g/km', 'Ft': 'fuel_type',
    'ec (cm3)': 'engine_size_cm3', 'Fuel consumption ': 'fuel_consumption_l/100km',
    'year': 'year_of_fabrication', 'Mh': 'manufacturer', 'z (Wh/km)': 'energy_consumption_Wh/km',
    'ep (KW)': 'electric_power_KW', 'Mk': 'model', 'Cn': 'commercial_name',
    'Electric range (km)': 'electric_range_km', 'MMS': 'manufacturar_name'
}

# scan_csv (Lazy) - Esto NO carga el archivo en RAM
df_lazy = (
    pl.scan_csv('ignore_data.csv', infer_schema_length=10000) # En base a las primeras 10k filas, Polars infiere el tipo de cada columna.
    .select(columns_to_load)
    .rename({k: v for k, v in rename_columns.items() if k in columns_to_load})
    .drop_nulls(subset=['model', 'commercial_name', 'fuel_type', 'year_of_fabrication'])
    .with_columns(cs.string().fill_null("").str.strip_chars().str.to_lowercase())
)

print("Iniciando procesamiento en streaming...")
df_lazy.sink_parquet(f"ignore_data_complete_clean.parquet") #
print("Done!")

file_weight = os.path.getsize("ignore_data_complete_clean.parquet")
print('.parquet size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

duckdb.sql("SUMMARIZE SELECT * FROM 'ignore_data_complete_clean.parquet'").show(max_rows=100)

Iniciando procesamiento en streaming...
Done!
.parquet size:  0.4 GB
┌──────────────────────────┬─────────────┬─────────┬────────────────────────┬───────────────┬────────────────────┬───────────────────┬─────────┬──────────┬──────────┬──────────┬─────────────────┐
│       column_name        │ column_type │   min   │          max           │ approx_unique │        avg         │        std        │   q25   │   q50    │   q75    │  count   │ null_percentage │
│         varchar          │   varchar   │ varchar │        varchar         │     int64     │      varchar       │      varchar      │ varchar │ varchar  │ varchar  │  int64   │  decimal(9,2)   │
├──────────────────────────┼─────────────┼─────────┼────────────────────────┼───────────────┼────────────────────┼───────────────────┼─────────┼──────────┼──────────┼──────────┼─────────────────┤
│ ID                       │ BIGINT      │ 1       │ 162072500              │      42470147 │ 36048809.00429475  │ 43904659.75443747 │ 4117191 │ 10

---

Para eliminar

In [ ]:
# Obtenemos las columnas del Data Frame
df_columns = pd.read_csv('ignore_data.csv', nrows=0)
print('Columnas del data frame crudo: \n', df_columns.columns)

# Cargamos unicamente las que nos interesan
df_columns.drop(columns=['Enedc (g/km)', 'r', 'VFN', 'Mp', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Cr', 'Mt', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'IT', 'Ernedc (g/km)', 'De','Vf', 'Status', 'ech', 'RLFI'], inplace=True)
columns_to_load = ['ID', 'year', 'Country', 'Mh', 'z (Wh/km)', 'ep (KW)', 'Mk', 'Cn', 'Electric range (km)', 'm (kg)', 'Ewltp (g/km)', 'Ft', 'ec (cm3)', 'Fuel consumption ', 'MMS']
# 'year', 'Country', 'Date of registration', 'Enedc (g/km)', 'r', 'VFN', 'Mp', 'Man', 'Tan', 'T', 'Va', 'Ve', 'MMS', 'Ct', 'Cr', 'Mt', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Fm', 'IT', 'Ernedc (g/km)', 'Erwltp (g/km)', 'De','Vf', 'Status', 'ech', 'RLFI', '

# Separamos la informacion en fragmentos de 1 millon para que la computadora no se sobresature
df_crud = pd.read_csv('ignore_data.csv', chunksize=1000000, usecols=columns_to_load)
print(f"\n\nColumnas que usaremos para el proyecto: \n", df_columns.columns)

In [ ]:
# Cambiamos de nombre algunas columnas
rename_columns = {'m (kg)': 'mass_kg', 'Ewltp (g/km)': 'co2_emission_g/km', 'Ft': 'fuel_type', 'ec (cm3)': 'engine_size_cm3', 'Fuel consumption ': 'fuel_consumption_l/100km', 'year': 'year_of_fabrication', 'Mh': 'manufacturer', 'z (Wh/km)': 'energy_consumption_Wh/km', 'ep (KW)': 'electric_power_KW', 'Mk': 'model', 'Cn': 'commercial_name', 'Electric range (km)': 'electric_range_km', 'MMS': 'manufacturar_name'}

parquet_dir = 'ignore_df_data_clean_parquets' 
os.makedirs(parquet_dir, exist_ok=True)

for i, chunk in enumerate(df_crud): 
    chunk = chunk.rename(columns=rename_columns)
    chunk = chunk.dropna(subset=['model', 'commercial_name', 'fuel_type', 'year_of_fabrication']) # Drop rows where all elements are NaN

    for col in chunk.select_dtypes(include=['object']).columns:  
        chunk[col] = chunk[col].astype(str).fillna('').str.lower().str.strip() # .apply(lambda x: x.lower().strip())

    chunk.to_parquet(
        f"{parquet_dir}/part_{i:04d}.parquet",
        engine='pyarrow',
        compression='snappy', 
        index=False
    )
    print(f"Chunk {i} processed and saved as parquet.")

del df_crud

---
---
---

In [13]:
# Cuantos tipos de combustible hay y cuantas filas por cada tipo de combustible
duckdb.sql("""
    SELECT fuel_type, COUNT(fuel_type) as total 
    FROM parquet_scan('ignore_data_complete_clean.parquet') 
    GROUP BY fuel_type 
    ORDER BY total DESC
""").show(max_rows=100)

┌─────────────────┬──────────┐
│    fuel_type    │  total   │
│     varchar     │  int64   │
├─────────────────┼──────────┤
│ petrol          │ 46169187 │
│ diesel          │ 23359519 │
│ electric        │  5191468 │
│ petrol/electric │  3459335 │
│ lpg             │  1309400 │
│ ng              │   224594 │
│ diesel/electric │   189870 │
│ e85             │   107578 │
│ ng-biomethane   │    52196 │
│ petrol-electric │     5829 │
│ hydrogen        │     4257 │
│ unknown         │     2801 │
│                 │     2674 │
│ cng             │     2476 │
│ diesel-electric │      988 │
│ biodiesel       │       93 │
│ hybrid/petrol/e │       44 │
│ petrol phev     │       18 │
│ other           │        9 │
│ gnl             │        2 │
│ ng_biomethane   │        1 │
│ petrol-gas      │        1 │
├─────────────────┴──────────┤
│ 22 rows          2 columns │
└────────────────────────────┘



<p style="font-size: 30px; text-align: center; color: #FF52A0;">Voy por acá</p>

---

In [14]:
#Cambiamos las unidades
df1['Engine_size_cm3'] = df1['Engine_size_cm3']/1000
df1.rename(columns={'Engine_size_cm3': 'Engine_size'}, inplace=True)

df1.to_csv('ignore_df_data_clean.csv')

df1

,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,56003309,1710.0,128.0,petrol,2.487,5.7
1,56003313,1710.0,128.0,petrol,2.487,5.7
2,56003314,1710.0,128.0,petrol,2.487,5.7
3,56003351,1635.0,160.0,petrol,1.987,7.1
4,56003352,1635.0,160.0,petrol,1.987,7.1
...,...,...,...,...,...,...
23869025,124068624,1395.0,135.0,petrol,1.498,6.0
23869026,124068626,1387.0,117.0,diesel,1.968,4.5
23869028,124068628,983.0,119.0,petrol,1.197,5.2
23869029,124068629,1665.0,129.0,petrol,2.487,5.7


In [15]:
#El auto que menos combustible consume es de 5.4 l/100km. Si hay autos que consumen menos es porque estan mal catalogados y son hibridos.
df1 = df1[df['Fuel_consumption_(l/100km)'] > 5.4]

#Los vehiculos mas livianos del mercado rondan los 900 kilos, y a su vez los mas pesados rondan los 3000 kilos
df1 = df1[(df1['Mass_(kg)'] >= 900) & (df['Mass_(kg)'] <= 3000)]

df1

C:\Users\matia\AppData\Local\Temp\ipykernel_22028\899938115.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df1[df['Fuel_consumption_(l/100km)'] > 5.4]
C:\Users\matia\AppData\Local\Temp\ipykernel_22028\899938115.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df1[(df1['Mass_(kg)'] >= 900) & (df['Mass_(kg)'] <= 3000)]


,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,56003309,1710.0,128.0,petrol,2.487,5.7
1,56003313,1710.0,128.0,petrol,2.487,5.7
2,56003314,1710.0,128.0,petrol,2.487,5.7
3,56003351,1635.0,160.0,petrol,1.987,7.1
4,56003352,1635.0,160.0,petrol,1.987,7.1
...,...,...,...,...,...,...
23869017,124068614,1967.0,167.0,diesel,1.969,6.4
23869019,124068617,1291.0,134.0,petrol,1.199,5.9
23869024,124068623,1395.0,139.0,petrol,1.199,6.1
23869025,124068624,1395.0,135.0,petrol,1.498,6.0


In [8]:
df1 = pd.read_csv('ignore_df_data_clean.csv')
print('Tamano del archivo df1:', (round((os.path.getsize('ignore_df_data_clean.csv')/1024)/1024, 2)), 'MB')
df1.shape

Tamano del archivo df1: 0.0 MB


(0, 19)

---

### <p style='color: green; text-align: center;'>Vamos a utilizar 1 millon de filas para que el archivo pese menos de 100 megas y poder subirlo a GitHub</p>

In [16]:
# Vamos a utilizar menos volumen de datos para poder maniobrar mas facil
df_aux = df1.sample(2500000) # 2.5 millon random rows
df_aux.to_csv('df_fit.csv', index=False)
print('Tamano del archivo df2:', round((os.path.getsize('df_fit.csv')/1024)/1024, 2), 'MB')

df2 = pd.read_csv('df_fit.csv')
print(df2.shape)
df2

Tamano del archivo df2: 96.09 MB
(2500000, 6)


,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,123030158,1295.0,132.0,petrol,1.199,5.9
1,50063074,1860.0,193.0,petrol,1.991,8.5
2,480784,1400.0,142.0,petrol,1.498,6.3
3,51827229,1395.0,132.0,petrol,1.499,5.8
4,79992271,1538.0,150.0,petrol,1.598,6.6
...,...,...,...,...,...,...
2499995,85681412,1296.0,131.0,petrol,0.999,5.8
2499996,90308953,1471.0,147.0,petrol,1.498,6.5
2499997,56852404,1720.0,131.0,petrol,2.487,5.8
2499998,124009753,1399.0,138.0,petrol,1.498,6.1


---

#### Convertir el archivo en html
<code>jupyter nbconvert --to html clean_data.ipynb</code>